In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime
import torch
import math

from modules.util.util import importstr
from modules.util.logconf import logging
log = logging.getLogger('nb')

In [3]:
def run(app, *argv):    
    torch.cuda.empty_cache()
    
    argv = list(argv)
    argv.insert(0, '--num-workers=4')
    log.info("Running: {}({!r}).main()".format(app, argv))
    
    app_cls = importstr(*app.rsplit('.', 1))
    app_cls(argv).main()
    
    log.info("Finished: {}.{!r}.main()".format(app, argv))

In [4]:
# width_irc=(16,128,128)
# run('prepcache.CovidPrepCacheApp', f'--batch-size={20}', '--width-irc', f'{width_irc[0]}', f'{width_irc[1]}', f'{width_irc[2]}')

2020-12-06 17:29:01,663 INFO     pid:44610 nb:006:run Running: prepcache.CovidPrepCacheApp(['--num-workers=4', '--batch-size=20', '--width-irc', '16', '128', '128']).main()
2020-12-06 17:29:02,911 INFO     pid:44610 prepcache:039:main Starting CovidPrepCacheApp, Namespace(batch_size=20, num_workers=4, width_irc=['16', '128', '128'])
100%|██████████| 10/10 [02:59<00:00, 17.95s/it]
2020-12-06 17:32:02,459 INFO     pid:44610 nb:011:run Finished: prepcache.CovidPrepCacheApp.['--num-workers=4', '--batch-size=20', '--width-irc', '16', '128', '128'].main()


## Basic Unet 3D

In [4]:
batch_size = 2
epochs = 50
steps_per_epoch=200
val_cadence = 5
recall_priority = 0
unet_depth=3
pad_types = ['zero', 'replicate']
windows = ['lung', 'mediastinal', 'shifted_lung']
project_name = 'covid19_seg'
width_irc = (12,192,192)

In [5]:
model_path = 'saved-models/2020-11-22_11.01.15.best.state'

In [ ]:
run_name = 'unet3d train run p2'

In [1]:
run('training.CovidSegmentationTrainingApp', f'--epochs={epochs}', f'--batch-size={batch_size}', 
    f'--steps-per-epoch={steps_per_epoch}', f'--val-cadence={val_cadence}', f'--recall-priority={recall_priority}',
    f'--depth={unet_depth}', f'--pad-type={pad_types[1]}', '--augmented', f'--model-path={model_path}',
    '--width-irc', f'{width_irc[0]}', f'{width_irc[1]}', f'{width_irc[2]}', 
    f'--project-name={project_name}', f'--run-name={run_name}')

# dummy run to finish wandb
run('training.CovidSegmentationTrainingApp', '--epochs=0', '--run-name=to delete')

## Covid Seg Net

In [6]:
batch_size = 2
epochs = 3
steps_per_epoch=160
val_cadence = 1
depth = 3
windows = ['lung', 'mediastinal', 'shifted_lung']
project_name = 'covid19_seg'
width_irc=(16,128,128)

In [5]:
run_name = 'CSN,1CLR,CropRandCrop,AllAug'
notes = 'Full train run on CSN showed that the model was evidently overfitting. \
This could be due to the fact that the dataloader was explicitly programmed to feed in data with lesions at the center. \
For this experiment, the full CT is initially cropped to the relevant area and 3 random subcrops are subsequently sampled, \
alternating with lesions or no lesions at the center. Already, the learning rate finder is showing a more promising learning rate curve. \
Additionaly, the dice loss and CE loss were weighted equally but the model seemed to be favoring the CE loss. For this run, a weighting of .7 and .3 is \
applied respectively. Hypothesis: random crops in the relevant area should provide more regularization than focused crops with augmentation'

In [ ]:
run('training_alpha.CovidSegmentationTrainingApp', f'--epochs={epochs}', f'--batch-size={batch_size}', 
    f'--ct-window={windows[2]}', f'--val-cadence={val_cadence}', f'--depth={depth}', '--augmented',
    '--width-irc', f'{width_irc[0]}', f'{width_irc[1]}', f'{width_irc[2]}', f'--notes={notes}',
    f'--steps-per-epoch={steps_per_epoch}', f'--project-name={project_name}', f'--run-name={run_name}')


# dummy run to finish wandb
run('training.CovidSegmentationTrainingApp', '--epochs=0', '--run-name=to delete')

2020-12-06 19:03:03,969 INFO     pid:53380 nb:006:run Running: training_alpha.CovidSegmentationTrainingApp(['--num-workers=4', '--epochs=3', '--batch-size=2', '--ct-window=shifted_lung', '--val-cadence=1', '--depth=3', '--augmented', '--width-irc', '16', '128', '128', '--notes=Full train run on CSN showed that the model was evidently overfitting. This could be due to the fact that the dataloader was explicitly programmed to feed in data with lesions at the center. For this experiment, the full CT is initially cropped to the relevant area and 3 random subcrops are subsequently sampled, alternating with lesions or no lesions at the center. Already, the learning rate finder is showing a more promising learning rate curve. Additionaly, the dice loss and CE loss were weighted equally but the model seemed to be favoring the CE loss. For this run, a weighting of .7 and .3 is applied respectively. Hypothesis: random crops in the relevant area should provide more regularization than focused cro

2020-12-06 19:03:07,270 INFO     pid:53380 training_alpha:217:init_model Using CUDA; 1 devices.
2020-12-06 19:03:09,396 INFO     pid:53380 modules.dsets:374:__init__ TrainingV2Covid2dSegmentationDataset: training mode, 159 uid's, 160 steps_per_epoch
2020-12-06 19:03:09,397 INFO     pid:53380 modules.dsets:447:__init__ TrainingV2Covid2dSegmentationDataset: (16, 128, 128) width_irc
2020-12-06 19:03:09,399 INFO     pid:53380 modules.dsets:374:__init__ Covid2dSegmentationDataset: validation mode, 40 uid's, 40 steps_per_epoch
2020-12-06 19:03:09,405 INFO     pid:53380 training_alpha:516:main Starting CovidSegmentationTrainingApp, Namespace(augment_flip=False, augment_noise=False, augment_offset=False, augment_rotate=False, augment_scale=False, augmented=True, batch_size=2, ct_window='shifted_lung', depth=3, epochs=3, model_path=None, notes='Full train run on CSN showed that the model was evidently overfitting. This could be due to the fact that the dataloader was explicitly programmed to fe

epoch,loss/trn,dice_loss/trn,ce_loss/trn,loss/val,dice_loss/val,ce_loss/val,metrics_val/miss_rate,metrics_val/fp_to_mask_ratio,pr_val/precision,pr_val/recall,pr_val/f1_score
1,0.7929,0.9005,0.5417,0.8284,0.9898,0.4518,0.7806,38.5962,0.0057,0.2194,0.0110
2,0.6786,0.8005,0.3942,0.6531,0.7890,0.3360,0.6883,0.5214,0.3741,0.3117,0.3401
3,0.6273,0.7363,0.3731,0.6144,0.7355,0.3318,0.5731,0.6848,0.3840,0.4269,0.4043


2020-12-06 19:04:35,373 INFO     pid:53380 training_alpha:384:log_metrics E1 CovidSegmentationTrainingApp
2020-12-06 19:04:35,374 INFO     pid:53380 training_alpha:415:log_metrics E1 trn      0.7929 loss, 0.9005 dice loss, 0.5417 ce loss, 0.0729 precision, 0.4551 recall, 0.1257 f1 score 0.5449 miss rate 5.7842 fp to label ratio
2020-12-06 19:08:51,603 INFO     pid:53380 training_alpha:384:log_metrics E1 CovidSegmentationTrainingApp
2020-12-06 19:08:51,604 INFO     pid:53380 training_alpha:426:log_metrics E1 val      0.8284 loss, 0.9898 dice loss, 0.4518 ce_loss, 0.0057 precision, 0.2194 recall, 0.0110 f1 score 0.7806 miss rate 38.5962 fp to label ratio
2020-12-06 19:08:53,748 INFO     pid:53380 training_alpha:502:save_model Saved model params to saved-models/2020-12-06_19.03.05.160.state
2020-12-06 19:08:53,755 INFO     pid:53380 training_alpha:507:save_model Saved model params to saved-models/2020-12-06_19.03.05.best.state
2020-12-06 19:08:53,766 INFO     pid:53380 training_alpha:512: